In [4]:
import csv
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

# Setup stealth Chrome
options = uc.ChromeOptions()
options.add_argument("--window-size=1920,1080")

driver = uc.Chrome(options=options, headless=False)

# Target URL
url = "https://www.realestate.com.au/buy/property-house-townhouse-unit+apartment-villa-between-0-550000-in-perth+-+cbd+and+inner+suburbs,+wa/list-1?misc=ex-under-contract&source=refinement"
driver.get(url)
time.sleep(5)

# Extract listings
cards = driver.find_elements(By.CLASS_NAME, "residential-card__content-wrapper")
print(f"🔍 Found {len(cards)} listings")

# Prepare CSV
with open("perth_properties.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Address", "Link", "Bedrooms", "Bathrooms", "Parking", "Area", "Type"])

    for card in cards:
        try:
            address_el = card.find_element(By.CLASS_NAME, "residential-card__address-heading")
            address = address_el.text
            link = address_el.find_element(By.TAG_NAME, "a").get_attribute("href")
            full_link = "https://www.realestate.com.au" + link

            features = card.find_elements(By.CSS_SELECTOR, "ul.residential-card__primary li")
            beds = baths = cars = area = ""
            for f in features:
                label = f.get_attribute("aria-label").lower()
                if "bedroom" in label:
                    beds = label
                elif "bathroom" in label:
                    baths = label
                elif "car space" in label:
                    cars = label
                elif "m²" in label or "building size" in label:
                    area = label

            try:
                prop_type = card.find_element(By.CSS_SELECTOR, "ul.residential-card__primary p").text
            except:
                prop_type = ""

            writer.writerow([address, full_link, beds, baths, cars, area, prop_type])
        except Exception as e:
            print(f"❌ Error parsing card: {e}")

driver.quit()
print("📄 Scraping complete. Data saved to perth_properties.csv")

🔍 Found 25 listings
📄 Scraping complete. Data saved to perth_properties.csv
